In [ ]:
# from src.data import get_data, DF
# from src.models import MLP

import pandas as pd

import torch.optim as optim
import torch.nn as nn
import torch.utils.data as data
import torch

from src.models import MLP, train, evaluate, epoch_time, MyDataset

import time, copy

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
lr = 0.000703113446890762
weight_decay = 0.00006156146508441857
# select = 'Ridge+GA'
select = 'manual+10'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
recipes = list(range(10))
# recipes = [0,3,7]
dataset_train = MyDataset(pd.read_csv(f"data/select/{select}_train.csv"), recipes)
dataset_test = MyDataset(pd.read_csv(f"data/select/{select}_test.csv"), recipes)
test_size = int(0.5 * len(dataset_test))
test_data, valid_data = torch.utils.data.random_split(
    dataset_test, [len(dataset_test) - test_size, test_size]
)
BATCH_SIZE = 128
train_iterator = data.DataLoader(dataset_train,shuffle=True,batch_size=BATCH_SIZE)
valid_iterator = data.DataLoader(valid_data,batch_size=BATCH_SIZE)
test_iterator = data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [ ]:
model = MLP(3 * len(recipes), [72, 39, 10], 3, 'tanh').to(device)
# model.load_state_dict(torch.load(f"./{select}-model.pt"))
# best_model = copy.deepcopy(model)
optimizer = optim.Adam(
    model.parameters(),
    lr=lr,
    weight_decay=weight_decay
)
# criterion = nn.MSELoss()
criterion = nn.L1Loss()
criterion = criterion.to(device)

In [ ]:
EPOCHS = 100000

best_valid_loss = float('inf')

for epoch in range(EPOCHS):

    start_time = time.monotonic()

    train_loss = train(model, train_iterator, optimizer, criterion, device)
    valid_loss = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'models/{select}.pt')
        best_model = copy.deepcopy(model)

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s | Train Loss: {train_loss:.5f} | Val. Loss: {valid_loss:.5f}')